<a href="https://colab.research.google.com/github/Edriczz/Formal_logic/blob/main/Edge_Coloring_with_Z3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Edge Coloring
there are not only proper vertex coloring on graph, there also edge coloring on graph that we will determine each **edge chromatic number** on a graph.


In [4]:
# prompt: install z3 solver library

!pip install z3-solver

In [5]:
from z3 import *

In [13]:
def convert_adj_matrix_to_graph_data(adj_matrix):
    """
    Converts an adjacency matrix to the number of vertices and a list of edges,
    treating the graph as simple (ignoring multi-edges and multiple loops).
    Args:
        adj_matrix (list of list of int): The adjacency matrix.
                                          adj_matrix[i][j] > 0 means an edge exists between i and j.
                                          Counts > 1 are treated as 1 (simple graph).
                                          For undirected graphs, it should be symmetric.
                                          Loops are adj_matrix[i][i] > 0.
    Returns:
        tuple (int, list): Number of vertices, list of edge tuples for the simple graph.
                           e.g., if adj_matrix[0][1]=2, one edge (0,1) is added.
    Raises:
        ValueError: If matrix is not square or contains negative entries.
    """
    num_vertices = len(adj_matrix)
    if num_vertices == 0:
        return 0, []

    # Validate square matrix
    for i, row in enumerate(adj_matrix):
        if len(row) != num_vertices:
            raise ValueError(f"Adjacency matrix must be square. Row {i} has length {len(row)}, expected {num_vertices}.")

    edges = []
    # Iterate upper triangle (j >= i) to process each potential edge/loop once
    for i in range(num_vertices):
        for j in range(i, num_vertices):
            count = adj_matrix[i][j]
            if count < 0:
                raise ValueError(f"Adjacency matrix entry adj_matrix[{i}][{j}] cannot be negative: {count}")

            # For undirected graphs, matrix should be symmetric in terms of edge existence.
            # The actual counts might differ if original was a multigraph, but we only care about existence.
            if i != j and (adj_matrix[i][j] > 0) != (adj_matrix[j][i] > 0):
                print(f"Warning: Adjacency matrix implies non-symmetric edge existence for ({i},{j}). "
                      f"adj_matrix[{i}][{j}]={adj_matrix[i][j]}, adj_matrix[{j}][{i}]={adj_matrix[j][i]}. "
                      f"Using upper triangle: edge ({i},{j}) exists if adj_matrix[{i}][{j}] > 0.")
            elif i !=j and adj_matrix[i][j] != adj_matrix[j][i] and (adj_matrix[i][j] > 0 and adj_matrix[j][i] > 0) :
                 print(f"Note: Adjacency matrix has different counts for edge ({i},{j}) "
                      f"adj_matrix[{i}][{j}]={adj_matrix[i][j]}, adj_matrix[{j}][{i}]={adj_matrix[j][i]}. "
                      f"Interpreting as a single edge since multigraphs are ignored.")


            # If adj_matrix[i][j] > 0, an edge (or loop if i==j) exists. Add it once.
            if count > 0:
                edges.append((i, j))
    return num_vertices, edges


In [15]:
def get_max_incident_edges(adj_matrix):
    """
    Calculates the maximum number of edges incident to any single vertex in a graph
    represented by an adjacency matrix, treating it as a simple graph.
    This is Delta (max degree) of the simple graph.
    A loop (adj_matrix[i][i] > 0) counts as one incident edge to vertex i for coloring.
    Args:
        adj_matrix (list of list of int): The adjacency matrix. Values > 1 are treated as 1.
    Returns:
        int: The maximum degree (Delta) of the interpreted simple graph.
    """
    num_vertices = len(adj_matrix)
    if num_vertices == 0:
        return 0

    max_degree = 0
    for i in range(num_vertices):
        current_vertex_degree = 0
        for j in range(num_vertices):
            if adj_matrix[i][j] > 0: # An edge (i,j) or loop (i,i) exists
                current_vertex_degree += 1
        if current_vertex_degree > max_degree:
            max_degree = current_vertex_degree

    return max_degree

In [16]:
def solve_edge_coloring_from_adj_matrix(adj_matrix, k_colors):
    """
    Tries to find a k-edge-coloring for the graph (from adj_matrix) using Z3.
    Args:
        adj_matrix (list of list of int): The adjacency matrix.
        k_colors (int): The number of available colors.
    Returns:
        list or None: A list of colors corresponding to the order of edges
                      derived from adj_matrix if a k-coloring exists, otherwise None.
    """
    num_vertices, edges = convert_adj_matrix_to_graph_data(adj_matrix)

    if not edges: # No edges to color
        return []

    s = Solver()
    # Create a color variable for each edge instance.
    edge_vars = [Int(f"edge_{i}_color") for i in range(len(edges))]

    # Constraint 1: Each edge's color must be between 1 and k_colors.
    for i in range(len(edges)):
        s.add(And(edge_vars[i] >= 1, edge_vars[i] <= k_colors))

    # Constraint 2: Edges sharing a common vertex must have different colors.
    for v_idx in range(num_vertices):
        incident_edge_color_vars = []
        for edge_idx, (u, v_node) in enumerate(edges):
            # An edge (u, v_node) is incident to v_idx if v_idx is one of its endpoints.
            if u == v_idx or v_node == v_idx:
                incident_edge_color_vars.append(edge_vars[edge_idx])

        if len(incident_edge_color_vars) > 1:
            s.add(Distinct(incident_edge_color_vars))

    if s.check() == sat:
        model = s.model()
        coloring_list = [model[edge_vars[i]].as_long() for i in range(len(edges))]
        return coloring_list
    else:
        return None

In [17]:
def find_edge_chromatic_number_from_adj_matrix(adj_matrix):
    """
    Finds the edge chromatic number of a graph defined by an adjacency matrix.
    Args:
        adj_matrix (list of list of int): The adjacency matrix.
    Returns:
        tuple (int, list): (edge_chromatic_number, list_of_colored_edges)
                           list_of_colored_edges is [(edge_tuple, color), ...].
                           Returns (0, []) for an empty graph.
                           Returns (-1, []) if no coloring found in search range.
    """
    num_vertices, edge_list_for_output = convert_adj_matrix_to_graph_data(adj_matrix)

    if not edge_list_for_output:
        return 0, [] # No edges, 0 colors needed.

    # Lower bound for k is the maximum number of edges incident to any single vertex.
    start_k = get_max_incident_edges(adj_matrix)

    # If start_k is 0 but there are edges, something is wrong, but convert_... should ensure edges exist if start_k > 0
    # If graph has edges, start_k must be at least 1.
    if start_k == 0 and edge_list_for_output: # Should ideally not happen if graph has edges
        start_k = 1

    # Search for the minimum number of colors k.
    # Upper bound: number of edges in the graph + 1 (for the range).
    # len(edge_list_for_output) is the total number of edges (counting multiedges).
    for k in range(start_k, len(edge_list_for_output) + 2):
        print(f"Attempting to edge-color with {k} colors...")
        coloring_result_list = solve_edge_coloring_from_adj_matrix(adj_matrix, k)

        if coloring_result_list is not None:
            print(f"Successfully found an edge coloring with {k} colors.")
            # Pair the derived edge_list_for_output with the found colors
            final_coloring_representation = []
            if len(edge_list_for_output) != len(coloring_result_list):
                # This should not happen if logic is correct
                print("Error: Mismatch between number of edges and number of colors returned.")
                return -1, []
            for i in range(len(edge_list_for_output)):
                final_coloring_representation.append( (edge_list_for_output[i], coloring_result_list[i]) )
            return k, final_coloring_representation
        else:
            print(f"Failed to find an edge coloring with {k} colors.")

    print("Could not determine the edge chromatic number within the search range.")
    return -1, []

In [20]:
if __name__ == '__main__':
    # --- Example Test Cases ---

    # 1. Triangle graph (C3)
    # Expected chromatic number: 3 (Max incident edges = 2. Odd cycle => Delta + 1)
    print("\n--- Example 1: Triangle Graph (C3) ---")
    adj_c3 = [
        [0, 1, 1],
        [1, 0, 1],
        [1, 1, 0]
    ]
    try:
        chi_prime_c3, coloring_c3 = find_edge_chromatic_number_from_adj_matrix(adj_c3)
        print(f"Edge chromatic number for C3: {chi_prime_c3}")
        if chi_prime_c3 != -1:
            print("Coloring:")
            for edge, color in coloring_c3:
                print(f"Edge {edge}: Color {color}")
    except ValueError as e:
        print(f"Error: {e}")





--- Example 1: Triangle Graph (C3) ---
Attempting to edge-color with 2 colors...
Failed to find an edge coloring with 2 colors.
Attempting to edge-color with 3 colors...
Successfully found an edge coloring with 3 colors.
Edge chromatic number for C3: 3
Coloring:
Edge (0, 1): Color 1
Edge (0, 2): Color 2
Edge (1, 2): Color 3


In [23]:
    print("\n--- Example Graph ---")
    adj_xyz =[
        #  1  2  3  4  5  6
        [0, 1, 0, 1, 0, 0],  # vertex 1: connected to 2, 4
        [1, 0, 1, 1, 1, 0],  # vertex 2: connected to 1, 3, 4, 5
        [0, 1, 0, 1, 1, 1],  # vertex 3: connected to 2, 4, 5, 6
        [1, 1, 1, 0, 1, 0],  # vertex 4: connected to 1, 2, 3, 5
        [0, 1, 1, 1, 0, 1],  # vertex 5: connected to 2, 3, 4, 6
        [0, 0, 1, 0, 1, 0]   # vertex 6: connected to 3 , 5
    ]
    try:
        chi_prime_xyz, coloring_xyz = find_edge_chromatic_number_from_adj_matrix(adj_xyz)
        print(f"Edge chromatic number for xyz: {chi_prime_xyz}")
        if chi_prime_xyz != -1:
            print("Coloring:")
            for edge, color in coloring_xyz:
                print(f"Edge {edge}: Color {color}")
    except ValueError as e:
        print(f"Error: {e}")


--- Example Graph ---
Attempting to edge-color with 4 colors...
Successfully found an edge coloring with 4 colors.
Edge chromatic number for xyz: 4
Coloring:
Edge (0, 1): Color 4
Edge (0, 3): Color 3
Edge (1, 2): Color 2
Edge (1, 3): Color 1
Edge (1, 4): Color 3
Edge (2, 3): Color 4
Edge (2, 4): Color 1
Edge (2, 5): Color 3
Edge (3, 4): Color 2
Edge (4, 5): Color 4
